In [ ]:
import pandas as pd, numpy as np
import sys,os,shutil,gc,re,json,glob,math,time,random,warnings
from tqdm import tqdm


In [ ]:
DIR_PROCESSED_DATA = os.environ['DIR_PROCESSED_DATA']
# DIR_PROCESSED_DATA = '/home/jovyan/digilut/data/processed'
DIR_TILES = f'{DIR_PROCESSED_DATA}/workspace/train/tiles'
DIR_TRAIN_CROPS = f'{DIR_PROCESSED_DATA}/p3_crops_p_100_500'
DIR_TRAIN_DATASET_OUT =  f'{DIR_PROCESSED_DATA}/p3'
os.makedirs(DIR_TRAIN_DATASET_OUT,exist_ok=True)

MIN_QUALITY_SCORE = 0.1

In [ ]:
df = pd.read_pickle(f'{DIR_TRAIN_CROPS}/anno.pkl').rename(columns={'fileid':'file_id','slideid':'slide_id'})
df['path'] = DIR_TRAIN_CROPS + '/' + df.file_id
df['category_id'] = 0
df['bbid'] = df.og_bbox.astype(str)
df.describe(percentiles=[i/10 for i in range(10)])

In [ ]:
def do_boxes_intersect(x1, y1, x2, y2, X1, Y1, X2, Y2):
    if x1 < X2 and x2 > X1 and y1 < Y2 and y2 > Y1:
        return True
    else:
        return False


def load_neg(base_sz):
    paths = glob.glob(f'{DIR_TILES}/{base_sz}/*/*/img/*')
    df_neg = pd.DataFrame(dict(path=paths))
    df_neg['slide_id'] = df_neg.path.str.split('/').str[-3]
    df_neg['sz'] = df_neg.path.str.split('/').str[-4]
    df_neg['file_id'] = df_neg.slide_id+'_'+df_neg.sz+'_'+df_neg.path.str.split('/').str[-1]
    df_neg['tile_id'] = df_neg.path.str.split('/').str[-1].str.split('_').str[0].astype(np.int32)
    d1 = df_neg.file_id.str.split('.').str[0].str.rsplit('_',n=3,expand=True).drop(columns=1)
    d1.columns = ['slide_id','y','x']
    # df_res['slide_id'] = d1.slide_id
    df_neg['x1'] = d1.x.astype(int)
    df_neg['y1'] = d1.y.astype(int)

    paths=glob.glob(f'{DIR_TILES}/{base_sz}/*/*/*.csv')
    lens=[]
    d_tiles = []
    notiles=[]
    for p in paths:
        try:
            d=pd.read_csv(p)
            lens.append(len(d))
            d_tiles.append(d)
        except:
            notiles.append(p)

    d_tiles=pd.concat(d_tiles).sort_values(by=['slide_id','tile_id']).reset_index(drop=True)
    d_tiles = d_tiles.rename(columns={'w':'twidth','h':'theight'})
    df_neg = pd.merge(df_neg,d_tiles,on=['slide_id','tile_id'])
    df_neg['base_sz'] = base_sz
    df_neg = df_neg[df_neg.Score>=MIN_QUALITY_SCORE]

    intersects = []
    for _,row in tqdm(df_neg.iterrows(),total=len(df_neg)):
        inter = False
        tile_bounds = [row.x1,row.y1,row.x1+row.twidth,row.y1+row.theight]
        X1,Y1,X2,Y2 = tile_bounds
        slide_bbs = df[df.slide_id==row.slide_id].full_bbox.tolist()
        for bb in slide_bbs:
            x1,y1,x2,y2 = bb
            inter = do_boxes_intersect(x1, y1, x2, y2, X1, Y1, X2, Y2)
            if inter:
                break #no need to continue checking

        intersects.append(inter)
    # d_tiles = d_tiles[['slide_id','tile_id','twidth','theight','swidth','sheight']]
    # d_tiles['x1'] =

    df_neg['has_bb'] = intersects
    display(df_neg.has_bb.value_counts())
    df_neg = df_neg[~df_neg.has_bb].reset_index(drop=True)
    return df_neg
dfs = []
# for base_sz in [128]:
for base_sz in [128,160,192,224,256]:
    df_neg = load_neg(base_sz)
    dfs.append(df_neg)

df_neg = pd.concat(dfs).reset_index(drop=True)
df_neg

In [ ]:
meta = pd.concat([df,df_neg]).reset_index(drop=True)
meta = meta.drop_duplicates(subset=['file_id'])
meta.shape

In [27]:
out_dir = f'{DIR_TRAIN_DATASET_OUT}/images'
os.makedirs(out_dir,exist_ok=True)
for file_name,d in tqdm(meta.groupby('file_id')):
    path = d.path.values[0]
    # im = cv2.imread(path)
    # height,width,_ = im.shape
    shutil.copy2(path, f'{out_dir}/{file_name}')


100%|██████████| 2678/2678 [00:00<00:00, 4857.77it/s]


In [28]:
df_neg.to_pickle(f'{DIR_TRAIN_DATASET_OUT}/neg.pkl')

In [29]:
df.to_pickle(f'{DIR_TRAIN_DATASET_OUT}/anno.pkl')